### Step1. Crawl the petiton page

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
result = pd.DataFrame()

for i in range(584274, 595226):
  URL = "http://www1.president.go.kr/petitions/"+str(i)

  response = requests.get(URL) #request 보내고 response 받기
  html = response.text
  soup = BeautifulSoup(html, 'html.parser') #HTML 코드에 데이터 추출하는 모듈
                                            #html.parser로 HTML 코드를 파이썬이 이해할 수 있도록 객체 구조 변환
                                            #즉, parsing이란 원하는 데이터를 특정 패턴으로 가공하는 것
  title = soup.find('h3', class_='petitionsView_title')
  count = soup.find('span', class_='counter')  

  for content in soup.select('div.petitionsView_write > div.View_write'):
      content

  a = []
  for tag in soup.select('ul.petitionsView_info_list > li'): 
      a.append(tag.contents[1])   

  if len(a) != 0:
    df1=pd.DataFrame({ 'start' : [a[1]],                
                        'end' : [a[2]],                     
                        'category' :  [a[0]],               
                        'count' : [count.text],             
                        'title': [title.text],              
                        'content': [content.text.strip()[0:13000]]                              
                      })
    
    result=pd.concat([result, df1]) #누적 병합                      
    result.index = np.arange(len(result))       

    if i % 60 == 0:                                        
      print("Sleep 90seconds. Count:" + str(i)           
            +",  Local Time:"+ time.strftime('%Y-%m-%d', time.localtime(time.time()))
            +" "+ time.strftime('%X', time.localtime(time.time()))
            +",  Data Length:"+ str(len(result)))        
      time.sleep(90)   

In [ ]:
print(result.shape)

df = result
df.head()

In [ ]:
df.loc[1]['content']
'''
'\n', '\r'과 같은 문자로 공백이 표현되어 있음
따라서 개행 문자나 특수 문자는 글 맥락에 영향을 주지 않으며
토크나이징 및 임베팅 시 성능 저하를 일으킬 수 있어 제거
'''

### Step1-1. Pre-processing
 

In [ ]:
import re #정규 표현식 지원 모듈

def remove_white_space(text):
  text = re.sub(r'[\t\r\n\f\v]', '', str(text))
  return text
'''
re.sub(): [\t\r\n\f\v] 이런 문자가 들어올 경우, 공백으로 치환해 text로 반환
'''

def remove_special_char(text): #아래 문자에 해당하지 않은 것이 들어온 경우 공백으로 교체
                                #한국어 word embedding vector 생성 위해 영어도 공백으로 교체
  text = re.sub('[^ ㄱ-ㅣ가-힣 0-9]+', '', str(text))
  return text

df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)